### CitiBike Data Extraction from online database

In [17]:
import pandas as pd
import requests 
import zipfile
import io

In [18]:
# URLs for June 2013 and June 2019 data (Citi Bike archive)
urls = {
    "2013": "https://s3.amazonaws.com/tripdata/2013-citibike-tripdata.zip",
    "2019": "https://s3.amazonaws.com/tripdata/2019-citibike-tripdata.zip"
}

In [19]:
# fucntion to download and save the 2013 and 2019 ZIP file 
# download the file in chunck to prevent memory overload 
# comment out the below funtion to prevent memeory overload 

"""def download_zip(url, filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
                print(f'Downloaded: {filename}')
    else:
        print(f'Failed to download {filename}')

# download both files
for year, url in urls.items():
    download_zip(url, f"{year}-citibike-tripdata.zip")"""

'def download_zip(url, filename):\n    response = requests.get(url, stream=True)\n    if response.status_code == 200:\n        with open(filename, \'wb\') as file:\n            for chunk in response.iter_content(chunk_size=1024):\n                file.write(chunk)\n                print(f\'Downloaded: {filename}\')\n    else:\n        print(f\'Failed to download {filename}\')\n\n# download both files\nfor year, url in urls.items():\n    download_zip(url, f"{year}-citibike-tripdata.zip")'